<a href="https://colab.research.google.com/github/Asif1310/3009/blob/main/Cricketanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.mkdir('/content/json_data') # to create a directory to store all json files

In [2]:
# downloading zip file from the URL and extracting it to the new directory created in previous step
import zipfile 
!wget -O '/content/json_data/main.zip' 'https://cricsheet.org/downloads/t20s_json.zip'
zip1 = zipfile.ZipFile('/content/json_data/main.zip','r')
zip1.extractall('/content/json_data')
zip1.close()

--2022-07-11 07:39:24--  https://cricsheet.org/downloads/t20s_json.zip
Resolving cricsheet.org (cricsheet.org)... 64.90.49.16
Connecting to cricsheet.org (cricsheet.org)|64.90.49.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6632998 (6.3M) [application/zip]
Saving to: ‘/content/json_data/main.zip’

/content/json_data/ 100%[===================>]   6.33M  4.83MB/s    in 1.3s    

2022-07-11 07:39:26 (4.83 MB/s) - ‘/content/json_data/main.zip’ saved [6632998/6632998]



In [3]:
# removing the main ZIP file and a README.txt from the directory
!rm /content/json_data/README.txt
!rm /content/json_data/main.zip

In [4]:
import json
import pandas as pd
import numpy as np

#initializing an empty dataframe
df = pd.DataFrame(columns = ['city','date','venue', 'gender','team_1','team_2','toss_winner','toss_decision','winner','target','margin','victory_margin','MOM','referee','tv_umpire','field_umpire_1','field_umpire_2'])


In [5]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  g = '/content/json_data/' + i
  data = json.load(open(g))
  try:
    df.loc[j,'city'] = data['info']['city']
  except KeyError:
    df.loc[j,'city'] = np.NaN
  df.loc[j,'date'] = data['info']['dates'][0]
  try:
    df.loc[j,'winner'] = data['info']['outcome']['winner'] 
  except KeyError: 
    try:
      df.loc[j,'winner'] = data['info']['outcome']['eliminator'] #tie-eliminator
    except KeyError:
      df.loc[j,'winner'] = data['info']['outcome']['result'] # no result
  try:
    df.loc[j,'margin'] = str(data['info']['outcome']['by'])
  except KeyError:
    try:
      if data['info']['outcome']['result'] == 'tie':
        df.loc[j,'margin'] = 'Tie-eliminator'
      elif data['info']['outcome']['result'] == 'no result':
        df.loc[j,'margin'] = np.NaN
    except KeyError:
      df.loc[j,'margin'] = data['info']['outcome']['method']
  try:
    df.loc[j,'MOM'] = data['info']['player_of_match'][0]
  except KeyError:
    df.loc[j,'MOM'] = np.NaN
  try:
    df.loc[j,'referee'] = data['info']['officials']['match_referees'][0]
  except KeyError:
    df.loc[j,'referee'] = np.NaN
  try:
    df.loc[j,'tv_umpire'] = data['info']['officials']['tv_umpires'][0]
  except KeyError:
    df.loc[j,'tv_umpire'] = np.NaN
  try:
    df.loc[j,'field_umpire_1'] = data['info']['officials']['umpires'][0]
  except KeyError:
    df.loc[j,'field_umpire_1'] = np.NaN
  try:
    df.loc[j,'field_umpire_2'] = data['info']['officials']['umpires'][1]
  except KeyError:
    df.loc[j,'field_umpire_2'] = np.NaN
  list1 = []
  for key, value in data['info']['players'].items():
    list1.append(key)
  df.loc[j, 'team_1'] = list1[0]
  df.loc[j, 'team_2'] = list1[1]
  df.loc[j, 'gender'] = data['info']['gender']
  df.loc[j, 'toss_winner'] = data['info']['toss']['winner']
  df.loc[j, 'toss_decision'] =  data['info']['toss']['decision']
  df.loc[j, 'venue'] = data['info']['venue']
  try:
    df.loc[j, 'target'] = data['innings'][1]['target']['runs']
  except KeyError:
    df.loc[j, 'target'] = np.NaN
  except IndexError:
    df.loc[j, 'target'] = np.NaN
  data.clear()



In [6]:
df['margin'] = df['margin'].str.strip('{}') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace(':','') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace('\'','') # transforming dictionaries in margin column to a proper string format
df_new = df['margin'].str.split(' ', expand = True) 
df_new[0] = df_new[0].str.replace('\'','')
df['victory_margin'] = df_new[1] + ' ' + df_new[0] # assigning transformed strings to the victory_margin column
df.drop(['margin'],axis = 1, inplace = True) #dropping the redundant margin column


In [7]:
df.head()

,city,date,venue,gender,team_1,team_2,toss_winner,toss_decision,winner,target,victory_margin,MOM,referee,tv_umpire,field_umpire_1,field_umpire_2
0,Cardiff,2015-08-31,Sophia Gardens,female,Australia,England,England,field,England,112,5 wickets,NR Sciver,DT Jukes,RT Robinson,RJ Bailey,RK Illingworth
1,NaN,2015-11-30,Sharjah Cricket Stadium,male,England,Pakistan,England,bat,England,155,NaN,Shoaib Malik,RS Madugalle,Shozab Raza,Ahmed Shahab,Ahsan Raza
2,Sharjah,2021-10-26,Sharjah Cricket Stadium,male,New Zealand,Pakistan,Pakistan,field,Pakistan,135,5 wickets,Haris Rauf,RS Madugalle,P Wilson,MA Gough,RA Kettleborough
3,Chittagong,2014-03-29,Zahur Ahmed Chowdhury Stadium,male,Netherlands,New Zealand,New Zealand,field,New Zealand,152,6 wickets,BB McCullum,DC Boon,SJ Davis,Aleem Dar,BNJ Oxenford
4,Al Amarat,2022-02-12,Al Amerat Cricket Ground Oman Cricket (Ministr...,male,Oman,Ireland,Ireland,field,Ireland,138,9 wickets,A Balbirnie,Muhammad Javed,NaN,Rahul Asher,Vinod Babu


In [8]:
df.dtypes

city              object
date              object
venue             object
gender            object
team_1            object
team_2            object
toss_winner       object
toss_decision     object
winner            object
target            object
victory_margin    object
MOM               object
referee           object
tv_umpire         object
field_umpire_1    object
field_umpire_2    object
dtype: object

In [9]:
df_players = pd.DataFrame(columns = ['venue','date','T1','T2']) # A new dataframe to store playing XI details

In [10]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  list1 = []
  player_list = []
  g = '/content/json_data/' + i
  data = json.load(open(g))
  df_players.loc[j, 'venue'] = data['info']['venue']
  df_players.loc[j,'date'] = data['info']['dates'][0]
  for key, value in data['info']['players'].items():
    list1.append(key)
    for player in value:
      player_list.append(player)
  df_players.loc[j,'T1P1'] = player_list[0]
  df_players.loc[j,'T1P2'] = player_list[1]
  df_players.loc[j,'T1P3'] = player_list[2]
  df_players.loc[j,'T1P4'] = player_list[3]
  df_players.loc[j,'T1P5'] = player_list[4]
  df_players.loc[j,'T1P6'] = player_list[5]
  df_players.loc[j,'T1P7'] = player_list[6]
  df_players.loc[j,'T1P8'] = player_list[7]
  df_players.loc[j,'T1P9'] = player_list[8]
  df_players.loc[j,'T1P10'] = player_list[9]
  df_players.loc[j,'T1P11'] = player_list[10]
  df_players.loc[j,'T2P1'] = player_list[11]
  df_players.loc[j,'T2P2'] = player_list[12]
  df_players.loc[j,'T2P3'] = player_list[13]
  df_players.loc[j,'T2P4'] = player_list[14]
  df_players.loc[j,'T2P5'] = player_list[15]
  df_players.loc[j,'T2P6'] = player_list[16]
  df_players.loc[j,'T2P7'] = player_list[17]
  df_players.loc[j,'T2P8'] = player_list[18]
  df_players.loc[j,'T2P9'] = player_list[19]
  df_players.loc[j,'T2P10'] = player_list[20]
  df_players.loc[j,'T2P11'] = player_list[21]  
  df_players.loc[j, 'T1'] = list1[0]
  df_players.loc[j, 'T2'] = list1[1]

In [11]:
df_players.head()

,venue,date,T1,T2,T1P1,T1P2,T1P3,T1P4,T1P5,T1P6,...,T2P2,T2P3,T2P4,T2P5,T2P6,T2P7,T2P8,T2P9,T2P10,T2P11
0,Sophia Gardens,2015-08-31,Australia,England,MM Lanning,EJ Villani,JE Cameron,EA Perry,JL Jonassen,AJ Blackwell,...,DN Wyatt,SJ Taylor,NR Sciver,KH Brunt,LS Greenway,HC Knight,GA Elwiss,A Shrubsole,D Hazell,RL Grundy
1,Sharjah Cricket Stadium,2015-11-30,England,Pakistan,JJ Roy,JM Vince,JE Root,MM Ali,EJG Morgan,JC Buttler,...,Rafatullah Mohmand,Mohammad Hafeez,Shoaib Malik,Mohammad Rizwan,Umar Akmal,Shahid Afridi,Sohail Tanvir,Anwar Ali,Aamer Yamin,Mohammad Irfan
2,Sharjah Cricket Stadium,2021-10-26,New Zealand,Pakistan,MJ Guptill,DJ Mitchell,KS Williamson,JDS Neesham,DP Conway,GD Phillips,...,Babar Azam,Fakhar Zaman,Mohammad Hafeez,Shoaib Malik,Imad Wasim,Asif Ali,Shadab Khan,Hasan Ali,Haris Rauf,Shaheen Shah Afridi
3,Zahur Ahmed Chowdhury Stadium,2014-03-29,Netherlands,New Zealand,SJ Myburgh,MR Swart,W Barresi,PW Borren,TLW Cooper,BN Cooper,...,KS Williamson,BB McCullum,LRPL Taylor,CJ Anderson,JDS Neesham,L Ronchi,NL McCullum,TA Boult,KD Mills,MJ McClenaghan
4,Al Amerat Cricket Ground Oman Cricket (Ministr...,2022-02-12,Oman,Ireland,KH Prajapati,Jatinder Singh,Shoaib Khan,Ayan Khan,Zeeshan Maqsood,Khawar Ali,...,A Balbirnie,GJ Delany,C Campher,MR Adair,GH Dockrell,SC Getkate,J Little,CA Young,L Tucker,Simi Singh


In [12]:
list2 = []
for i in os.listdir('/content/json_data'): #looping through all the JSON files in the folder
  g = '/content/json_data/' + i
  data = json.load(open(g))
  date = data['info']['dates'][0]
  venue = data['info']['venue']
  for inning in data['innings']:
    team = inning['team']
    for over in (inning['overs']):
      over_num = over['over']
      ball_counter = 0
      for ball in over['deliveries']:
        list1 = []
        ball_counter += 1
        batter = ball['batter']
        bowler = ball['bowler']
        non_striker = ball['non_striker']
        batter_runs = ball['runs']['batter']
        extra_runs = ball['runs']['extras']
        total_runs = ball['runs']['total']
        team = team
        over_num = over_num
        try:
          for key,value in ball['extras'].items():
            extra_type = key
        except KeyError:
          extra_type = 'NA'
        try:
          wicket_type = ball['wickets'][0]['kind']
        except KeyError:
          wicket_type = 'NA'
        try:
          player_out = ball['wickets'][0]['player_out']
        except KeyError:
          player_out = 'NA'
        try:
          fielder = ball['wickets'][0]['fielders'][0]['name']
        except KeyError:
          fielder = 'NA'
        list1 = [date, venue, team, over_num, ball_counter, batter, bowler, non_striker,batter_runs,extra_runs,extra_type,total_runs,wicket_type,player_out,fielder]
        list2.append(list1)



In [13]:
df_main = pd.DataFrame(list2, columns = ['date', 'venue','team', 'over', 'ball_num','batter','bowler','non_striker','batter_runs','extra_runs','extra_type','total_runs', 'wicket_type', 'player_out', 'fielder'])

In [14]:
df_main[df_main['wicket_type']!= 'NA']

,date,venue,team,over,ball_num,batter,bowler,non_striker,batter_runs,extra_runs,extra_type,total_runs,wicket_type,player_out,fielder
8,2015-08-31,Sophia Gardens,Australia,1,3,MM Lanning,A Shrubsole,EJ Villani,0,0,NA,0,bowled,MM Lanning,NA
11,2015-08-31,Sophia Gardens,Australia,1,6,EJ Villani,A Shrubsole,JE Cameron,0,0,NA,0,caught,EJ Villani,LS Greenway
33,2015-08-31,Sophia Gardens,Australia,5,3,JE Cameron,A Shrubsole,EA Perry,0,0,NA,0,caught,JE Cameron,DN Wyatt
37,2015-08-31,Sophia Gardens,Australia,5,7,JL Jonassen,A Shrubsole,EA Perry,0,0,NA,0,caught,JL Jonassen,CM Edwards
69,2015-08-31,Sophia Gardens,Australia,10,6,EA Perry,NR Sciver,AJ Blackwell,0,0,NA,0,bowled,EA Perry,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474978,2015-11-22,Sheikh Zayed Stadium,Oman,18,2,Sultan Ahmed,Rohan Mustafa,Amir Ali,0,0,NA,0,caught,Sultan Ahmed,Zaheer Maqsood
474988,2015-11-22,Sheikh Zayed Stadium,Oman,19,6,AV Lalcheta,Qadeer Ahmed,Amir Ali,0,0,NA,0,run out,AV Lalcheta,Qadeer Ahmed
475021,2015-11-22,Sheikh Zayed Stadium,United Arab Emirates,5,1,Amjad Gul,AV Lalcheta,Rohan Mustafa,0,0,NA,0,bowled,Amjad Gul,NA
475066,2015-11-22,Sheikh Zayed Stadium,United Arab Emirates,12,4,Rohan Mustafa,Zeeshan Maqsood,Shaiman Anwar,0,0,NA,0,caught,Rohan Mustafa,Bilal Khan


In [15]:
df_main.dtypes

date           object
venue          object
team           object
over            int64
ball_num        int64
batter         object
bowler         object
non_striker    object
batter_runs     int64
extra_runs      int64
extra_type     object
total_runs      int64
wicket_type    object
player_out     object
fielder        object
dtype: object